In [ ]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [ ]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [ ]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

## Tensorboard writer for loss logging

In [ ]:
writer = SummaryWriter()

## GPU/CPU selection

In [ ]:
#device = torch.device("cuda:0")
device = torch.device("cpu")

## Reweighting strategy - must be min, mean or max

In [ ]:
reference_method = 'mean'

## Hyperparameters

In [ ]:
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 2  # Number of layers in every made network
n_made_units = 100 # Number of units in every layer of the made network
n_flow_layers = 6 # Number of layers in the flow

batch_size = 1024
n_epochs = 800
adam_lr = 0.001     # Learning rate for the ADAM optimizer (default: 0.001)

n_train = int(1e5)  # Number of training events
n_test = int(1e2)   # Number of testing events
n_sample = int(1e3) # Number of samples for ess evaluation

## Load the train data and reweight

In [ ]:
# Load
train_samples_preweighting = np.genfromtxt("data/weighted_samples.csv", delimiter=',')[:n_train]
train_weights_preweighting = np.genfromtxt("data/weighted_weights.csv", delimiter=',')[:n_train]

# Get reference weight
reference_methods = {'min': np.amin, 'mean': np.mean, 'max': np.amax}
ref_weight = reference_methods[reference_method](train_weights_preweighting)

# Reweighting
train_samples = None
train_weights = None
for sample, weight in zip(train_samples_preweighting, train_weights_preweighting):
    if weight < ref_weight:
        p_accept = weight/ref_weight
        if (np.random.rand() < p_accept):
            if train_samples is None:
                train_samples = sample
                train_weights = 1
            else:
                train_samples = np.vstack((train_samples, sample))
                train_weights = np.vstack((train_weights, 1))

    else:
        new_weight = weight/ref_weight
        if train_samples is None:
            train_samples = sample
            train_weights = new_weight
        else:
            train_samples = np.vstack((train_samples, sample))
            train_weights = np.vstack((train_weights, new_weight))

# Normalize weights
train_weights /= train_weights.mean()

# Convert to torch tensors
train_samples = torch.tensor(train_samples, dtype=torch.float32, device=device)
train_weights = torch.tensor(train_weights, dtype=torch.float32, device=device)

del train_samples_preweighting
del train_weights_preweighting
gc.collect()

## Load the test data

In [ ]:
test_samples = torch.tensor(np.genfromtxt("data/unweighted_samples.csv", delimiter=',')[:n_test], dtype=torch.float32, device=device)

## Set up the flow

In [ ]:
event_dim = train_samples.shape[1]
base_dist = BoxUniform(torch.zeros(event_dim), torch.ones(event_dim))

transforms = []
for _ in range(n_flow_layers):
    transforms.append(RandomPermutation(features=event_dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=event_dim, 
        hidden_features=n_made_units,
        num_bins=n_RQS_knots,
        num_blocks=n_made_layers-1,
        tails="constrained",
        use_residual_blocks=False
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters(), lr=adam_lr)

scheduler = MultiStepLR(optimizer, milestones=[350, 425, 500, 575, 650, 725, 800], gamma=0.5)

## Training

In [ ]:
data_size = train_samples.shape[0]
n_batches = m.ceil(data_size/batch_size)

data_size_validation = test_samples.shape[0]
n_batches_validate = m.ceil(data_size_validation/batch_size)

best_validation_loss = np.inf
best_ess = 0

for epoch in range(n_epochs):
    scheduler.step()
    
    permutation = torch.randperm(data_size, device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        samples_batch = train_samples[indices]
        weights_batch = train_weights[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -(flow.log_prob(inputs=samples_batch)*weights_batch).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        if batch%25 == 0:
            print("epoch = ", epoch, "batch = ", batch, "/", n_batches, "loss = ", cum_loss)
    
    writer.add_scalar("Loss_train", cum_loss, epoch)


    # ---------- Compute validation loss -----------
    validation_loss = 0
    for batch in range(n_batches_validate):
        batch_begin = batch*batch_size
        batch_end = min( (batch+1)*batch_size, data_size_validation-1 )
        samples_batch = test_samples[batch_begin:batch_end]
    
        with torch.no_grad():
            validation_loss = (validation_loss*batch - (flow.log_prob(samples_batch)).mean())/(batch+1)

    print("Validation loss = ", validation_loss.item())
    writer.add_scalar("Loss_test", validation_loss.item(), epoch)

    if validation_loss < best_validation_loss:
        torch.save(flow, "flow_model_weighted_best_validation.pt")
        best_validation_loss = validation_loss

    
    # ---------- Compute effective sample size ----------
    # generate samples and evaluate llhs
    with torch.no_grad():
        samples = flow.sample(n_sample)
        llhs = flow.log_prob(samples)

    # Store files
    np.savetxt("/tmp/samples_file.csv", samples.numpy(), delimiter=',')
    np.savetxt("/tmp/llhs_file.csv", np.exp(llhs.numpy()), delimiter=',')

    # Run the evaluator
    cmd = os.path.abspath(os.getcwd())+'/ME_VEGAS/compute_metrics_from_likelihoods /tmp/samples_file.csv /tmp/llhs_file.csv'
    b = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout
    lines = b.decode('ascii').split("\n")

    ess = float(lines[2].split(' ')[-1])
    
    print("Effective sample size = ", ess)
    writer.add_scalar("Effective_sample_size", ess, epoch)

    if ess > best_ess:
        torch.save(flow, "flow_model_weighted_best_ess.pt")
        best_ess = ess
        
torch.save(flow, "flow_model_weighted_final.pt")